## The dice game of threes

[StackExchange.](https://math.stackexchange.com/questions/4482277/the-dice-game-of-threes)

* Roll five six-sided dice. Threes count as zero.
* Set aside at least one die and reroll the rest.
* Repeat until all five dice are set aside.
* Try to minimize the expected total ("cost") of the five dice.

### Rolling strategy

At each stage, you would never set aside a higher die over a lower die. Therefore we can iterate over the dice from lowest to highest; the cost of setting aside $k$ dice is concave up. However, as we shall see, the cost of the remaining dice is concave down, so we do have to consider all $n$ possibilities. Fortunately, with only five dice at most this is not difficult for a computer.

### Inductive solution

From here, we can compute the solution using linear memoization.

* Base case: Playing with 0 dice always produces 0 total and 0 threes.
* Inductive case: Consider all possible sorted rolls of a pool of $n$ dice, following the strategy above. The expected cost of keeping the lowest $k$ dice is the sum of those dice plus the expected cost of playing the game with $n - k$ dice, which we already computed in a previous iteration. Then we can just pick the choice of $k$ that minimizes the expected cost.


In [1]:
%pip install icepool

import icepool
from icepool import Die, vectorize

# Maps pool size -> outcome distribution.
# Start with base case: 0 total, 0 threes.
results = [Die([vectorize(0, 0)])]

die = Die([vectorize(1, 0), vectorize(2, 0), vectorize(0, 1), vectorize(4, 0), vectorize(5, 0), vectorize(6, 0)])

def play_game(sorted_rolls):
    n = len(sorted_rolls)
    total, threes = 0, 0
    best_score = None
    best_result = None
    for k, (number, is_three) in enumerate(sorted_rolls, start=1):
        total += number
        threes += is_three
        this_score = results[n - k].marginals[0].mean() + total
        if best_score is None or this_score < best_score:
            best_score = this_score
            best_result = results[n - k] + vectorize(total, threes)
    return best_result

for n in range(1, 6):
    results.append(icepool.map(play_game, die.pool(n)))

for n, die in enumerate(results):
    print(n, 'dice:')
    print(f'Mean cost: {float(die.marginals[0].mean()):0.3f}')
    print(die)

0 dice:
Mean cost: 0.000
Die with denominator 1

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          0 |          0 |        1 | 100.000000% |


1 dice:
Mean cost: 3.000
Die with denominator 6

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          0 |          1 |        1 |  16.666667% |
|          1 |          0 |        1 |  16.666667% |
|          2 |          0 |        1 |  16.666667% |
|          4 |          0 |        1 |  16.666667% |
|          5 |          0 |        1 |  16.666667% |
|          6 |          0 |        1 |  16.666667% |


2 dice:
Mean cost: 4.389
Die with denominator 216

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          0 |          2 |       12 |   5.555556% |
|          1 |          1 |       24 |  11.111111% |
|          2 |          0 |       12 |   5.555556% |